<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/stigmatizing_labels_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import os
import re
import sys
import scipy
import Cython
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en import English
import numpy as np

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive/')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [3]:

%cd /content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Stigmatizing Labels/1_Data Prep
%ls

/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Stigmatizing Labels/1_Data Prep
1_stigmatizing_labels_word_embeddings.py
doubt_word_list_round_2.csv
stigma_expanded_misspellings.csv
stigmatizing_labels_descriptors_lexicon_stem_and_similar_round1.csv
stigmatizing_labels_lexicon_dev.ipynb
stigmatizing_labels_regex.ipynb
stigmatizing_word_list_round_1_dw_pruned.csv
stigma_word_list_round_2.csv
trig-vectors-phrase.bin
trig-vectors-phrase.txt


BigQuery

In [4]:
#Replace 'project_id' with your BigQuery project ID

from google.cloud import bigquery
client = bigquery.Client(project='detecting-bias-mimic-iii')

icd_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.diagnoses_icd
                LIMIT 1000''')

icd = client.query(icd_query).to_dataframe()



ICD codes

In [5]:


#Filter for ICD codes for 282.60-282.69, referring to sickle cell types w/wo crisis
#2824 for thalassemia w + w/o crisis (282.41-282.42)
#SCD: 2826,2824
#Chronic Pain 3382
#Opioid dependencies: 3040,3047 (combo),
#HIV/AIDS ^042$

icds_of_interest = icd[icd['ICD9_CODE'].str.contains('2826|2824|3040|3047|3382|^042$', na=False)]
# Issues here-- will cause more leading numbers
print(icds_of_interest.head())

icds_of_interest.info()

#patients_unique = icds_of_interest['SUBJECT_ID'].drop_duplicates()


     ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
784  176454       15872   139593        3     73382
889  196755       17664   123887        7     33829
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 784 to 889
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ROW_ID      2 non-null      Int64 
 1   SUBJECT_ID  2 non-null      Int64 
 2   HADM_ID     2 non-null      Int64 
 3   SEQ_NUM     2 non-null      Int64 
 4   ICD9_CODE   2 non-null      object
dtypes: Int64(4), object(1)
memory usage: 104.0+ bytes


Notes and patients load

In [6]:
note_query = ('''SELECT *
                FROM physionet-data.mimiciii_notes.noteevents''')
                #LIMIT 100000''')

NOTES = client.query(note_query).to_dataframe()

peek_notes = NOTES.head()
print(peek_notes)

## Patients

patient_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.patients''')

PATIENTS = client.query(patient_query).to_dataframe()

peek_patients = PATIENTS.head()
print(peek_patients)

biased_notes_patients_corpus = NOTES.merge(PATIENTS, on = 'SUBJECT_ID')

biased_notes_patients_corpus.info()
# Find unique chart types and decide to limit them
    # Remove Radiology, ECG, Respiratory, Echo notes
biased_notes_patients_corpus_filtered = biased_notes_patients_corpus[biased_notes_patients_corpus["CATEGORY"].str.contains("Radiology|ECG|Respiratory|Echo")==False]
# biased_notes_patients_corpus_filtered.to_csv("biased_notes_patients_corpus_filtered.csv")



    ROW_ID  SUBJECT_ID  HADM_ID   CHARTDATE           CHARTTIME  \
0  1338192        5799   161311  2190-05-11 2190-05-11 16:39:00   
1  1580995       24528   168104  2140-10-29 2140-10-29 06:39:00   
2  1743358        4790   112219  2168-01-13 2168-01-13 14:27:00   
3  1682650         403   111164  2101-12-07 2101-12-07 12:07:00   
4  2072729       31338   197604  2181-11-03 2181-11-03 13:14:00   

            STORETIME       CATEGORY DESCRIPTION   CGID  ISERROR  \
0 2190-05-11 17:11:00  Nursing/other      Report  20808     <NA>   
1 2140-10-29 06:41:00  Nursing/other      Report  16975     <NA>   
2 2168-01-13 14:38:00  Nursing/other      Report  18255     <NA>   
3 2101-12-07 12:11:00  Nursing/other      Report  15444     <NA>   
4 2181-11-03 13:17:00  Nursing/other      Report  14943     <NA>   

                                                TEXT  
0  NPN\n\nPt's condition has progressive declined...  
1  resp care\nPt remains intubated & supported in...  
2  Neonatology-NNP Tria

Explode by sentence, regex quote match

In [7]:
full_dataframe = biased_notes_patients_corpus_filtered
# Tokenize by sentence
nlp = English()  # just the language with no model

nlp.add_pipe('sentencizer')
full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])
full_dataframe = full_dataframe.explode("Sentence", ignore_index=True)
full_dataframe.rename(columns={"Unnamed: 0": "ROW_ID_new"}, inplace=True)
full_dataframe.index.name = "Sentence ID"

full_dataframe['Sentence'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)



<ipython-input-7-1e5fd6deed48>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])


# Regex matching all stemwords + any semantically similar word identified

In [9]:


doubt_word_list = pd.read_csv("stigma_word_list_round_2.csv")


# Create regex which is each stem word + all similar_words identified in the word embeddings step, which will be used to search the charts
def group_and_create_regex(dataframe):
    grouped_data = dataframe.groupby('stem_word')['similar_word'].apply(list).reset_index()
    grouped_data['regex'] = grouped_data.apply(lambda row: '|'.join([re.escape(row['stem_word'])] + [re.escape(word) for word in row['similar_word']]), axis=1)
    return grouped_data

grouped_data = group_and_create_regex(doubt_word_list)
print(grouped_data)

# Create function that takes regex from each row and searches the charts
def identify_matching_strings(full_dataframe, doubt_word_list):
    matching_dataframes = []


    for index, row in doubt_word_list.iterrows():
        regex = row['regex']
        stem_word = row['stem_word']
        matches = full_dataframe.loc[full_dataframe['Sentence'].str.contains(regex, flags=re.IGNORECASE)].copy()
        matches['regex'] = regex
        matches['stem_word'] = stem_word
        matching_dataframes.append(matches)

    if matching_dataframes:
        return pd.concat(matching_dataframes, axis=0)
    else:
        return pd.DataFrame()


result = identify_matching_strings(full_dataframe, grouped_data)
result




         stem_word                                       similar_word  \
0           abuser  [ivdabuser, abuser's, abusers, abuse/abuser, a...   
1           addict  [exaddict, addict*, deaddict, methaddict, addi...   
2       aggressive       [.aggressive, ,aggressive, highlyaggressive]   
3         agitated  [o=agitated, agitatedx1, aagitated, s.agitated...   
4        alcoholic  [non-alcoholic/alcoholic, prealcoholic, .alcoh...   
5            blame                [blaming, blames, blameful, blamed]   
6      challenging  [challengingly, hallenging, challengin, challe...   
7        combative  [combatively, combatives, noncombative, combat...   
8         confront  [confronts, confronted, confronter, confrontin...   
9        defensive  [overdefensive, counterdefensive, defensive/of...   
10    drug-seeking  [non-drug-seeking, drug-seeking/drug-taking, '...   
11           drunk  [drunks, drunken, drunkest, drunker, drunkenly...   
12      exaggerate  [exaggerates, overexaggerate, e

,ROW_ID_x,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,Sentence,regex,stem_word
Sentence ID,,,,,,,,,,,,,,,,,,,,,
46207,15610,3133,135744,2173-08-11,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,2965,M,2131-02-13,2174-06-07,2174-06-07,2174-06-07,1,Admission Date: [**2173-7-5**] Discharge Date:...,abuser|ivdabuser|abuser's|abusers|abuse/abuser...,abuser
47056,15612,3133,179884,2174-02-02,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,2965,M,2131-02-13,2174-06-07,2174-06-07,2174-06-07,1,PAST MEDICAL HISTORY: Status post liver trans...,abuser|ivdabuser|abuser's|abusers|abuse/abuser...,abuser
47143,15612,3133,179884,2174-02-02,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,2965,M,2131-02-13,2174-06-07,2174-06-07,2174-06-07,1,Ethanol abuse in the past.,abuser|ivdabuser|abuser's|abusers|abuse/abuser...,abuser
48383,402830,11236,156425,2195-04-04,2195-04-04 15:42:00,2195-04-04 16:04:08,Nursing,Nursing Progress Note,16617,<NA>,...,10627,F,2112-02-19,2195-04-10,2195-04-10,2195-04-10,1,Social Work was consulted to evaluate for pot...,abuser|ivdabuser|abuser's|abusers|abuse/abuser...,abuser
48694,402685,11236,156425,2195-04-03,2195-04-03 00:13:00,2195-04-03 03:01:47,General,Generic Note,16586,<NA>,...,10627,F,2112-02-19,2195-04-10,2195-04-10,2195-04-10,1,", to keep the dressing on for 4-5 days (Plast...",abuser|ivdabuser|abuser's|abusers|abuse/abuser...,abuser
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28113671,675,63660,175783,2169-09-02,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,37057,M,2119-06-16,NaT,NaT,NaT,0,He did not lose consciousness and was not stun...,unpleasant|unpleasantries|unpleasant/annoying|...,unpleasant
28773829,41238,50648,102957,2108-05-28,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,33583,M,2056-03-15,2108-06-01,2108-06-01,2108-06-01,1,"Although having diarrhea is unpleasant, the la...",unpleasant|unpleasantries|unpleasant/annoying|...,unpleasant
28841932,1324268,4765,154020,2177-04-06,2177-04-06 18:22:00,2177-04-06 18:33:00,Nursing/other,Report,17255,<NA>,...,4511,F,1877-03-15,2178-05-18,2178-05-18,2178-05-18,1,"She does open eyes to unpleasant stimuli and,...",unpleasant|unpleasantries|unpleasant/annoying|...,unpleasant


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [11]:

quoted_dataframe = result

quoted_dataframe["stigmatizing_patient_label"] = ""
quoted_dataframe["annotator_comments"] = ""
# X matching sentences from total of 9,118,501 sentences from 366,002 charts
quoted_dataframe2 = quoted_dataframe.drop_duplicates(subset=['Sentence'])
quoted_dataframe2.to_csv("stigmatizing_labels_dataframe_annotate.csv")



Explore the Data


In [ ]:
from google.colab import runtime
runtime.unassign()